# URL Embedding Clustering

In [27]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.append(os.path.abspath(".."))

from hdbscan import HDBSCAN
from sklearn import metrics
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from urlembed.util.plotter import *
from urlembed.util.metrics import *
from urlembed.util.seqmanager import *

import plotly.plotly as py
from plotly.graph_objs import *
from __future__ import print_function
from plotly.tools import FigureFactory as FF

The crawling proccess has been done in two different ways:

- **No costraint**: the crawler follows a random outlink from all of the outlinks in a given page
- **List costraint**: the crawler follows a random outlink but only from the outlinks in "lists"

# WORD2VEC - NO-COSTRAINT
Word embedding alorithm. Word2vec is a two-layer neural net that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. 

Here we're loading all the files that the crawler has generated to train word2vec model.

See the [Dataset README](https://github.com/chrisPiemonte/url2vec/tree/master/dataset "Dataset") for further information.

In [2]:
nocostraint_path = os.getcwd() + "/../dataset/cs.illinois.edu_NoConstraint.words1000.depth10/"
nocostraint_urlmap_path = nocostraint_path + "urlsMap.txt"
nocostraint_seq_path = nocostraint_path + "sequenceIDs.txt"

Training the model. No need to keep everything in RAM so we're passing two generators.

**PARAMETERS**:

- **min_count**: ignore all words with total frequency lower than this
- **window**: is the maximum distance between the current and predicted word within a sentence
- **negative**: if > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). Default is 5. If set to 0, no negative samping is used

In [11]:
# because of generator
vocab_sequences_nc = get_sequences(nocostraint_seq_path)
train_sequences_nc = get_sequences(nocostraint_seq_path)

w2v_model_nc = Word2Vec(min_count=1, window=5, negative=5)
w2v_model_nc.build_vocab(vocab_sequences_nc)
w2v_model_nc.train(train_sequences_nc)

994570

### T-SNE
Applying t-SNE for dimensionality reduction. We need two dimensional vectors for visualization purposes.

In [4]:
nocostraint_urlmap = get_urlmap(nocostraint_urlmap_path)

# 100-dim vecs
wordvecs_nc = np.array([w2v_model[key] for key in nocostraint_urlmap], dtype="float64")

# URL labels
urls_nc = [nocostraint_urlmap[key] for key in nocostraint_urlmap]

# 2-dim vecs
tsne = TSNE(n_components=2)
twodim_wordvecs_nc = tsne.fit_transform(wordvecs_nc)

## Clustering - No-Costraint Word2Vec model

### DBSCAN
DBSCAN - Density-Based Spatial Clustering of Applications with Noise. Finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density.

**PARAMETERS**:

- **eps** : The maximum distance between two samples for them to be considered as in the same neighborhood.
- **min_samples** : The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.

In [6]:
dbscan = DBSCAN(eps=0.9, min_samples=4)
dbscan.fit(wordvecs_nc)
dbscan_labels_nc = dbscan.labels_

dbscan_colors_nc = [get_color(clust) for clust in dbscan_labels_nc]

print("Clusters found with DBSCAN:", len(set(dbscan_labels_nc)))
print ([label for label in set(dbscan_labels_nc)])
print()
print()

Clusters found with DBSCAN: 19
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, -1]




### DBSCAN Plot

In [ ]:
dbscan_data_nc = scatter_plot(twodim_wordvecs_nc, urls_nc, dbscan_colors_nc)
py.iplot(dbscan_data_nc, filename='Word Vectors Nocostraint - Scatter plot DBSCAN')

<div>
    <a href="https://plot.ly/~chrispolo/0" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/nc_wordvectors_scatter_plot_DBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:0"  src="https://plot.ly/embed.js" async></script>
</div>

### HDBSCAN
HDBSCAN - Hierarchical Density-Based Spatial Clustering of Applications with Noise. Performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection.

**PARAMETERS**:
- **min_cluster_size** : minimum nodes to form a cluster

In [7]:
hdbscan = HDBSCAN(min_cluster_size=6)
hdbscan_labels_nc = hdbscan.fit_predict(wordvecs_nc)

hdbscan_colors_nc = [get_color(clust) for clust in hdbscan_labels_nc]

print("Clusters found with HDBSCAN:", len(set(hdbscan_labels_nc)))
print([label for label in set(hdbscan_labels_nc)])
print()
print()

Clusters found with HDBSCAN: 13
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, -1]




### HDBSCAN Plot

In [ ]:
hdbscan_data_nc = scatter_plot(twodim_wordvecs_nc, urls_nc, hdbscan_colors_nc)
py.iplot(hdbscan_data_nc, filename='Word Vectors Nocostraint - Scatter plot HDBSCAN')

<div>
    <a href="https://plot.ly/~chrispolo/2" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/nc_wordvectors_scatter_plot_HDBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:2"  src="https://plot.ly/embed.js" async></script>
</div>

### K-MEANS
The first step is to initialize the algorithm by choosing K initial cluster centroid locations. This is typically done by randomly choosing K points from the input set. With these initial K centroids, the algorithm proceeds by repeating the following two main steps:

- Cluster Assignment - Here, each observation (e.g., each point in the data set) is assigned to a cluster centroid such that the WCSS objective function is minimized. This can often be translated to assigning each observation to the closest cluster centroid (which coincidentally minimizes WCSS for many distance metrics), though for some distance metrics and spaces this need not be the case.
- Update Centroids - After all of the input observations have been assigned to a cluster centroid, each centroid is re-computed. For each cluster, the new centroid is computed by averaging the observations that were assigned to it (e.g., computing the 'mean' of the observations).

These steps are repeated until the algorithm "converges".

**PARAMETERS**:

- **n_clusters**: number of clusters

In [8]:
kmeans = KMeans(n_clusters=15)
kmeans.fit(wordvecs_nc)

kmeans_labels_nc = kmeans.labels_

kmeans_colors_nc = [get_color(clust) for clust in kmeans_labels_nc]

print("Clusters found with K-MEANS:", len(set(kmeans_labels_nc)))
print([label for label in set(kmeans_labels_nc)])

Clusters found with K-MEANS: 15
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


### K-MEANS Plot

In [ ]:
kmeans_data_nc = scatter_plot(twodim_wordvecs_nc, urls_nc, kmeans_colors_nc)
py.iplot(kmeans_data_nc, filename='Word Vectors Nocostraint - Scatter plot K-MEANS')

<div>
    <a href="https://plot.ly/~chrispolo/4" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/nc_wordvectors_scatter_plot_KMEANS.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:4"  src="https://plot.ly/embed.js" async></script>
</div>

### GROUND TRUTH

In [10]:
gt = GroundTruth(os.getcwd() + "/../dataset/ground_truth/urlToMembership.txt")
ground_truth_nc = [int(gt.get_groundtruth(nocostraint_urlmap[key])) for key in nocostraint_urlmap]

real_colors_nc = [get_color(n) for n in ground_truth_nc]

print("Clusters found manually:", len(set(ground_truth_nc)))
print([label for label in set(ground_truth_nc)])

Clusters found manually: 14
[0, 1, 2, 3, 4, 6, 8, 10, 11, 12, 13, 14, 15, -1]


### GROUND TRUTH Plot

In [ ]:
groundtruth_data_nc = scatter_plot(twodim_wordvecs_nc, urls_nc, real_colors_nc)
py.iplot(groundtruth_data_nc, filename='Word Vectors Nocostraint - Scatter plot Ground Truth')

<div>
    <a href="https://plot.ly/~chrispolo/56" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/nc_wordvectors_scatter_plot_GROUNDTRUTH.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:56"  src="https://plot.ly/embed.js" async></script>
</div>

---

# WORD2VEC - LIST-COSTRAINT
Word embedding alorithm. Word2vec is a two-layer neural net that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. 

Here we're loading all the files that the crawler has generated to train word2vec model.

See the [Dataset README](https://github.com/chrisPiemonte/url2vec/tree/master/dataset "Dataset") for further information.

In [13]:
listcostraint_path = os.getcwd() + "/../dataset/cs.illinois.edu_ListConstraint.words1000.depth10/"
listcostraint_urlmap_path = listcostraint_path + "urlsMap.txt"
listcostraint_seq_path = listcostraint_path + "sequenceIDs.txt"

Training the model. No need to keep everything in RAM so we're passing two generators.

**PARAMETERS**:

- **min_count**: ignore all words with total frequency lower than this
- **window**: is the maximum distance between the current and predicted word within a sentence
- **negative**: if > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). Default is 5. If set to 0, no negative samping is used

In [14]:
# because of generator
vocab_seq_lc = get_sequences(listcostraint_seq_path)
train_seq_lc = get_sequences(listcostraint_seq_path)

word2vec_lc = Word2Vec(min_count=1, window=5, negative=5)
word2vec_lc.build_vocab(vocab_seq_lc)
word2vec_lc.train(train_seq_lc)

1000000

### T-SNE
Applying t-SNE for dimensionality reduction. We need two dimensional vectors for visualization purposes.

In [15]:
listcostraint_urlmap = get_urlmap(listcostraint_urlmap_path)

# 100-dim vecs
wordvecs_lc = np.array([word2vec_lc[key] for key in listcostraint_urlmap], dtype="float64")

# URL labels
urls_lc = [listcostraint_urlmap[key] for key in listcostraint_urlmap]

# 2-dim vecs
tsne = TSNE(n_components=2)
twodim_wordvecs_lc = tsne.fit_transform(wordvecs_lc)

### DBSCAN
DBSCAN - Density-Based Spatial Clustering of Applications with Noise. Finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density.

**PARAMETERS**:

- **eps** : The maximum distance between two samples for them to be considered as in the same neighborhood.
- **min_samples** : The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.

In [16]:
dbscan = DBSCAN(eps=1.0, min_samples=5)
dbscan.fit(wordvecs_lc)
dbscan_labels_lc = dbscan.labels_

dbscan_colors_lc = [get_color(clust) for clust in dbscan_labels_lc]

print("Clusters found with DBSCAN:", len(set(dbscan_labels_lc)))
print ([label for label in set(dbscan_labels_lc)])
print()
print()

Clusters found with DBSCAN: 16
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, -1]




### DBSCAN Plot

In [ ]:
dbscan_data_lc = scatter_plot(twodim_wordvecs_lc, urls_lc, dbscan_colors_lc)
py.iplot(dbscan_data_lc, filename='Word Vectors Lists - DBSCAN')

<div>
    <a href="https://plot.ly/~chrispolo/60" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/lc_wordvectors_scatter_plot_DBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:60"  src="https://plot.ly/embed.js" async></script>
</div>

### HDBSCAN
HDBSCAN - Hierarchical Density-Based Spatial Clustering of Applications with Noise. Performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection.

**PARAMETERS**:
- **min_cluster_size** : minimum nodes to form a cluster

In [17]:
hdbscan = HDBSCAN(min_cluster_size=5)
hdbscan_labels_lc = hdbscan.fit_predict(wordvecs_lc)

hdbscan_colors_lc = [get_color(clust) for clust in hdbscan_labels_lc]

print("Clusters found with HDBSCAN:", len(set(hdbscan_labels_lc)))
print([label for label in set(hdbscan_labels_lc)])
print()
print()

Clusters found with HDBSCAN: 18
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, -1]




### HDBSCAN Plot

In [ ]:
hdbscan_data_lc = scatter_plot(twodim_wordvecs_lc, urls_lc, hdbscan_colors_lc)
py.iplot(hdbscan_data_lc, filename='Word Vectors Lists - Scatter plot HDBSCAN')

<div>
    <a href="https://plot.ly/~chrispolo/62" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/lc_wordvectors_scatter_plot_HDBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:62"  src="https://plot.ly/embed.js" async></script>
</div>

### K-MEANS
The first step is to initialize the algorithm by choosing K initial cluster centroid locations. This is typically done by randomly choosing K points from the input set. With these initial K centroids, the algorithm proceeds by repeating the following two main steps:

- Cluster Assignment - Here, each observation (e.g., each point in the data set) is assigned to a cluster centroid such that the WCSS objective function is minimized. This can often be translated to assigning each observation to the closest cluster centroid (which coincidentally minimizes WCSS for many distance metrics), though for some distance metrics and spaces this need not be the case.
- Update Centroids - After all of the input observations have been assigned to a cluster centroid, each centroid is re-computed. For each cluster, the new centroid is computed by averaging the observations that were assigned to it (e.g., computing the 'mean' of the observations).

These steps are repeated until the algorithm "converges".

**PARAMETERS**:

- **n_clusters**: number of clusters

In [19]:
kmeans = KMeans(n_clusters=15)
kmeans.fit(wordvecs_lc)

kmeans_labels_lc = kmeans.labels_

kmeans_colors_lc = [get_color(clust) for clust in kmeans_labels_lc]

print("Clusters found with K-MEANS:", len(set(kmeans_labels_lc)))
print([label for label in set(kmeans_labels_lc)])

Clusters found with K-MEANS: 15
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


### K-MEANS Plot

In [ ]:
kmeans_data_lc = scatter_plot(twodim_wordvecs_lc, urls_lc, kmeans_colors_lc)
py.iplot(kmeans_data_lc, filename='Word Vectors Lists - Scatter plot K-MEANS')

<div>
    <a href="https://plot.ly/~chrispolo/64" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/lc_wordvectors_scatter_plot_KMEANS.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:64"  src="https://plot.ly/embed.js" async></script>
</div>

### GROUND TRUTH

In [21]:
gt = GroundTruth(os.getcwd() + "/../dataset/ground_truth/urlToMembership.txt")
ground_truth_lc = [int(gt.get_groundtruth(listcostraint_urlmap[key])) for key in listcostraint_urlmap]

real_colors_lc = [get_color(n) for n in ground_truth_lc]

print("Clusters found manually:", len(set(ground_truth_lc)))
print([label for label in set(ground_truth_lc)])

Clusters found manually: 13
[0, 1, 2, 3, 4, 6, 8, 10, 12, 13, 14, 15, -1]


### GROUND TRUTH Plot

In [ ]:
groundtruth_data_lc = scatter_plot(twodim_wordvecs_lc, urls_lc, real_colors_lc)
py.iplot(groundtruth_data_lc, filename='Word Vectors Lists - Scatter plot Ground Truth')

<div>
    <a href="https://plot.ly/~chrispolo/66" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/lc_wordvectors_scatter_plot_GROUNDTRUTH.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:66"  src="https://plot.ly/embed.js" async></script>
</div>

---

# EVLUATION
Evaluating the performance of a clustering algorithm is not as trivial as counting the number of errors or the precision and recall of a supervised classification algorithm. In particular any evaluation metric should not take the absolute values of the cluster labels into account but rather if this clustering define separations of the data similar to some ground truth set of classes or satisfying some assumption such that members belong to the same class are more similar that members of different classes according to some similarity metric.

See the [scikit-learn documentaion](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation "ti") for futher information

### Metrics:
- **Adjusted Rand index**: Given the knowledge of the *ground truth* class assignments *true_label* and our clustering algorithm assignments of the same samples *pred_walktrap_label*, the adjusted Rand index is a function that measures the similarity of the two assignments, ignoring permutations and with chance normalization


- **Mutual Information based scores**: Given the knowledge of the ground truth class assignments labels_true and our clustering algorithm assignments of the same samples labels_pred, the Mutual Information is a function that measures the agreement of the two assignments, ignoring permutations. Two different normalized versions of this measure are available, Normalized Mutual Information(NMI) and Adjusted Mutual Information(AMI). NMI is often used in the literature while AMI was proposed more recently and is normalized against chance


- **Homogeneity**: each cluster contains only members of a single class


- **completeness**: all members of a given class are assigned to the same cluster


- **V-measure**: The V-measure is actually equivalent to the mutual information (NMI) discussed above normalized by the sum of the label entropies


In [34]:
metrics_df = pd.DataFrame([
        [
            # dbscan nocostraint
            metrics.homogeneity_score(ground_truth_nc, dbscan_labels_nc),
            metrics.completeness_score(ground_truth_nc, dbscan_labels_nc),
            metrics.v_measure_score(ground_truth_nc, dbscan_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, dbscan_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, dbscan_labels_nc)
        ],
        [
            # hdbscan nocostraint
            metrics.homogeneity_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.completeness_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.v_measure_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, hdbscan_labels_nc)
        ],
        [
            # kmeans nocostraint
            metrics.homogeneity_score(ground_truth_nc, kmeans_labels_nc),
            metrics.completeness_score(ground_truth_nc, kmeans_labels_nc),
            metrics.v_measure_score(ground_truth_nc, kmeans_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, kmeans_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, kmeans_labels_nc)
        ],
        [
            # dbscan listcostraint
            metrics.homogeneity_score(ground_truth_lc, dbscan_labels_lc),
            metrics.completeness_score(ground_truth_lc, dbscan_labels_lc),
            metrics.v_measure_score(ground_truth_lc, dbscan_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, dbscan_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, dbscan_labels_lc)
        ],
        [
            # hdbscan listcostraint
            metrics.homogeneity_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.completeness_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.v_measure_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, hdbscan_labels_lc)
        ],
        [
            # kmeans listcostraint
            metrics.homogeneity_score(ground_truth_lc, kmeans_labels_lc),
            metrics.completeness_score(ground_truth_lc, kmeans_labels_lc),
            metrics.v_measure_score(ground_truth_lc, kmeans_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, kmeans_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, kmeans_labels_lc)
        ]],
        index=["NoCostraint - DBSCAN", "NoCostraint - HDBSCAN", "NoCostraint - K-MEANS", "ListCostraint - DBSCAN", "ListCostraint - HDBSCAN", "ListCostraint - K-MEANS"],
        columns=["Homogeneity", "Completeness", "V-Measure core", "Adjusted Rand index", "Mutual Information"])

metrics_df

,Homogeneity,Completeness,V-Measure core,Adjusted Rand index,Mutual Information
NoCostraint - DBSCAN,0.555319,0.657997,0.602313,0.448750,0.523399
NoCostraint - HDBSCAN,0.575965,0.672053,0.620310,0.528249,0.552570
NoCostraint - K-MEANS,0.808513,0.742417,0.774056,0.777703,0.726263
ListCostraint - DBSCAN,0.416341,0.592220,0.488945,0.225089,0.393520
ListCostraint - HDBSCAN,0.476015,0.506710,0.490883,0.227523,0.451538
ListCostraint - K-MEANS,0.795348,0.640251,0.709422,0.592015,0.627247
